Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 1: Set Covering

First lab + peer review. List this activity in your final report, it will be part of your exam.

## Task

Given a number $N$ and some lists of integers $P = (L_0, L_1, L_2, ..., L_n)$, 
determine, if possible, $S = (L_{s_0}, L_{s_1}, L_{s_2}, ..., L_{s_n})$
such that each number between $0$ and $N-1$ appears in at least one list

$$\forall n \in [0, N-1] \ \exists i : n \in L_{s_i}$$

and that the total numbers of elements in all $L_{s_i}$ is minimum. 

## Instructions

* Create the directory `lab1` inside the course repo (the one you registered with Andrea)
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)
* Use `problem` to generate the problems with different $N$
* In the `README.md`, report the the total numbers of elements in $L_{s_i}$ for problem with $N \in [5, 10, 20, 100, 500, 1000]$ and the total number on $nodes$ visited during the search. Use `seed=42`.
* Use `GitHub Issues` to peer review others' lab

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.

**Deadline**

* Sunday, October 16th 23:59:59 for the working solution
* Sunday, October 23rd 23:59:59 for the peer reviews

In [1]:
import random
import logging

### Problem representation:
We need the least amount of numers (given some lists), that cover all the numbers $[0 - N]$:

\begin{equation}
min \sum \limits _{j=1} ^{n} c_{j}x_{j}
\end{equation}

where $c_{j}$ is the cost of the given list and 

\begin{equation}
x_{j} \ for \ set \ S_{j} \ = 
  \begin{cases}
    1 & \text{if $S_{j}$ selected}\\
    0 & \text{if not}\\
  \end{cases} 
\end{equation}

The cost of adding a list to the solution is the number of elements in the list. As priority of node expansion we use the length of the list itself. 

#### Constraints
Given the problem code, we can see that:
* Lists are of length $[N//5, N//2]$
* The number of lists to explore is between $[N, N*5]$

**Linear programming problem contraints:**
* Every element in the solution space must be covered. *This constraint cannot be guaranteed.*
* Values for $x_{j} \in \{0, 1\}$ 
**NP-hard optimization problem.** 

In [2]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

##### 
**Professor's Greedy solution**: 
Greedy solution considering length of list as *maximum expected value*.

In [3]:
import logging


def greedy(N):
    goal = set(range(N))
    covered = set()
    solution = list()
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    while goal != covered:
        x = all_lists.pop(0)
        if not set(x) < covered:
            solution.append(x)
            covered |= set(x)

    logging.info(
        f"Greedy solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )
    logging.debug(f"{solution}")

In [4]:
logging.getLogger().setLevel(logging.INFO)
for N in [5, 10, 20, 100, 500, 1000]:
    greedy(N)

INFO:root:Greedy solution for N=5: w=5 (bloat=0%)
INFO:root:Greedy solution for N=10: w=13 (bloat=30%)
INFO:root:Greedy solution for N=20: w=46 (bloat=130%)
INFO:root:Greedy solution for N=100: w=332 (bloat=232%)
INFO:root:Greedy solution for N=500: w=2162 (bloat=332%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)


In [5]:
%timeit greedy(1_000)

INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)


1.09 s ± 37.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### **Informed Solution**: A* Algorithm

Runs best first (Greedy approach), but adding estimated heuristic cost.

In [24]:
import functools

# Priority function takes into account the cost to get to given state
# and the heuristic estimated cost: considering the amount of elements missing from solution

# state_dim = number of elements in the state considered
# N = total number of numbers
# pot_coverage = potential coverage given the state
def priority_function(state_dim, N, pot_coverage):
    g_n = state_dim

    # Heuristic: number of numbers missing elements from Universal set
    h_n = N - pot_coverage
    res = g_n + h_n
    #print(f"Priority = {g_n} +  {h_n}")

    # Priority given by the (num of elements in current solution if list were to be added to it + num of missing elemets) first 
    # and then by (num of elements in current solution if list were to be added)
    return res, g_n

In [28]:
def get_state_dim(state):
    res= functools.reduce(lambda count, l: count + len(l), state, 0)
    #print(f"STATE (dim = {res}): {state}")
    return res

def get_coverage_len(state, goal):
    cov = set()
    for l in state:
        cov |= set(l)
    return len(cov)

In [30]:
from gx_utils import PriorityQueue


def a_star(N):
    goal = set(range(N))
    covered = set()
    state = list()
    problem_lists = problem(N, seed=42)
    unique_lists = set(tuple(i) for i in problem_lists)
    sorted_lists = sorted(unique_lists, key=lambda l: len(l))
    pq = PriorityQueue()
    
    # Insert all lists as tuples in a Priority Queue
    for l in sorted_lists:
        # Calculate the potential coverage if that list were to be put in the solution
        state.append(list(l))
        pot_coverage = get_coverage_len(state, goal)

        # Add to pq based on their length (at this stage the total cost considering heuristic will be = N)
        pq.push(l, p=priority_function(len(l), N, pot_coverage))
        state.pop()
    

    while covered != goal:
        # Extract most promising element from queue (optimal solution hp => it is the best element)
        #print(f"Current state: {state}")
        new_list = list(pq.pop())
        if not set(new_list) < covered:
            # Update current state and coverage
            state.append(new_list)
            covered |= set(new_list)
            
            # Revisit priorities: given the new state
            revisited_lists = list()
            while pq:
                revisited_lists.append(pq.pop())
            
            # Sort lists again based on length (Greedy)
            #revisited_lists = sorted(revisited_lists, key=lambda l: len(l))
            # Set potential coverage to current one
            #print(f"Covered before {covered}")

            # Recompute priorities
            for l in revisited_lists:
                # Re create pq with new priorities
                state.append(list(l))
                pot_coverage = get_coverage_len(state, goal)
                state_dim = get_state_dim(state)
                pq.push(l, p=priority_function(state_dim, N, pot_coverage))
                state.pop()
        #print(f"Covered final {covered}")


    logging.info(
        f"A* solution for N={N}: w={sum(len(_) for _ in state)} (bloat={(sum(len(_) for _ in state)-N)/N*100:.0f}%)"
    )
    logging.debug(f"{state}")
    
        
        

In [33]:
logging.getLogger().setLevel(logging.INFO)
for N in [5, 10, 20, 100, 500, 1000]:
    a_star(N)

INFO:root:A* solution for N=5: w=5 (bloat=0%)
INFO:root:A* solution for N=10: w=13 (bloat=30%)
INFO:root:A* solution for N=20: w=28 (bloat=40%)
INFO:root:A* solution for N=100: w=228 (bloat=128%)
INFO:root:A* solution for N=500: w=1828 (bloat=266%)
INFO:root:A* solution for N=1000: w=4130 (bloat=313%)


In [228]:
def add_one_list(current_state, goal, sorted_lists, visited_nodes, covered):
    if covered == goal:
        print(type(current_state), type(visited_nodes))
        return current_state, visited_nodes
    if len(sorted_lists) == 0:
        return current_state, -1

    new_list = list(sorted_lists.pop())
    current_state.append(new_list)
    covered |= set(new_list)
    visited_nodes += 1
    add_one_list(current_state, goal, sorted_lists
    , visited_nodes, covered)

        

In [229]:
from gx_utils import PriorityQueue

def a_star_recursive(N):
    goal = set(range(N))
    covered = set()
    state = list()
    prob_lists = problem(N, seed=42)
    unique_lists = set(tuple(i) for i in prob_lists)
    sorted_lists = sorted(unique_lists, key=lambda l: len(l))

    print(sorted_lists)

    for l in sorted_lists:
        # The first step takes the shortest list (at this stage all costs are equal, by definition)
        first_list = list(sorted_lists.pop(0))
        state.append(first_list)
        covered |= set(first_list)
        visited_nodes = 1
        solution, visited = add_one_list(state, goal, sorted_lists, visited_nodes, covered)
        
        if visited == -1:
            break

    if visited == -1:
       logging.info(f"A* found no solution for N={N}") 
    else:
        logging.info(
            f"A* solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in state)-N)/N*100:.0f}%)"
        )
        logging.debug(f"{state}")


In [230]:
logging.getLogger().setLevel(logging.INFO)
for N in [5]:
    a_star_recursive(N)

[(2,), (4,), (1,), (3,), (0,), (0, 1), (2, 4), (2, 3), (0, 2), (1, 3)]
<class 'list'> <class 'int'>


TypeError: cannot unpack non-iterable NoneType object